In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Config


In [ ]:
import torch

In [ ]:
# Hyperparameters etc.
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 2e-5
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 32
TEST_BATCH_SIZE = 128
NUM_EPOCHS = 50
NUM_WORKERS = 2
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 240
PIN_MEMORY = True
LOAD_MODEL = False

# Utils


In [ ]:
import glob
import os
from pathlib import Path

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

In [ ]:
def save_checkpoint(model, optimizer, save_dir='runs/', file_name='my_checkpoint.pt'):
    print('=> Saving checkpoint')
    save_path = os.path.join(save_dir, file_name)
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(checkpoint, save_path)


def load_checkpoint(model, optimizer, lr, load_dir='runs/', file_name='my_checkpoint.pt'):
    print('=> Loading checkpoint')
    load_path = os.path.join(load_dir, file_name)
    checkpoint = torch.load(load_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    for params_group in optimizer.params_groups:
        params_group['lr'] = lr
        
        
def check_accuracy(loader, model, loss_fn, device='cuda'):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    loss = 0

    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = model(x)
            loss += loss_fn(preds, y).item() * x.size(0)
            
            preds = torch.sigmoid(preds)
            preds = (preds >= .5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
            
    loss = loss / len(loader.dataset)

    print(
        f'Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}\t'
        f'Dice score: {dice_score/len(loader)}\t'
        f'Loss: {loss}'
    )
    model.train()

    return num_correct/num_pixels*100, dice_score/len(loader), loss


def save_predictions_as_imgs(
    loader, model, folder='save_images/', device='cuda', num=5,
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > .5).float()
        torchvision.utils.save_image(
            preds, f'{folder}pred_{idx}.png'
        )
        torchvision.utils.save_image(y.unsqueeze(1), f'{folder}{idx}.png')
        
        if idx > num:
            break

    model.train()
    
    
def visualize(display_list):
    plt.figure(figsize=(15, 20))
    title = ['Input image', 'True mask', 'Predicted mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[1])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()
    
    
def show_predictions(image, mask, device='cuda'):
    model.eval()
    
    image = image.unsqueeze(0).to(device)
    pred_mask = model(image).squeeze()
    
    image = image.squeeze(0).cpu().numpy().transpose(1, 2, 0)
    mask = mask.numpy()
    pred_mask = pred_mask.detach().cpu().numpy()
    
    visualize([image, mask, pred_mask])
    
    model.train()

# Dataset



In [ ]:
import pandas as pd
import os
import albumentations as A
import matplotlib.pyplot as plt
from zipfile import ZipFile
from PIL import Image
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
# train data
train_zip = '../input/carvana-image-masking-challenge/train.zip'

with ZipFile(train_zip, 'r') as zip_:
    zip_.extractall()

In [ ]:
# train mask data
train_masks_zip = '../input/carvana-image-masking-challenge/train_masks.zip'

with ZipFile(train_masks_zip, 'r') as zip_:
    zip_.extractall()

In [ ]:
# train_csv data
train_csv_zip = '../input/carvana-image-masking-challenge/train_masks.csv.zip'

with ZipFile(train_csv_zip, 'r') as zip_:
    zip_.extractall()

In [ ]:
car_ids = []
paths = []
for dir_name, _, file_names in os.walk('./train'):
    for file_name in file_names:
        path = os.path.join(dir_name, file_name)
        paths.append(path)
        
        car_id = file_name.split('.')[0]
        car_ids.append(car_id)
        
d = {'id':car_ids, 'car_path':paths}
df = pd.DataFrame(data = d)
df = df.set_index('id')

In [ ]:
car_ids = []
paths = []
for dir_name, _, file_names in os.walk('./train_masks'):
    for file_name in file_names:
        path = os.path.join(dir_name, file_name)
        paths.append(path)
        
        car_id = file_name.split('_mask')[0]
        car_ids.append(car_id)
        
d = {'id':car_ids, 'mask_path':paths}
mask_df = pd.DataFrame(data = d)
mask_df = mask_df.set_index('id')

In [ ]:
df['mask_path'] = mask_df['mask_path']

df

In [ ]:
class CarvanaDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        self.images = df['car_path'].tolist()
        self.masks = df['mask_path'].tolist()
        assert len(self.images) == len(self.masks), 'number of images data not equal to masks data'

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        mask_path = self.masks[idx]
        image = np.array(Image.open(img_path).convert('RGB'))
        mask = np.array(Image.open(mask_path).convert('L'), dtype=np.float32)
        mask[mask == 255.] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image = image, mask = mask)
            image = augmentations['image']
            mask = augmentations['mask']

        return image, mask

In [ ]:
# Augmentations

train_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=.1),
        A.VerticalFlip(p=.5),
        A.Normalize(
            mean=[0., 0., 0.],
            std=[1., 1., 1.],
            max_pixel_value=255.,
        ),
        ToTensorV2(),
    ]
)

val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0., 0., 0.],
            std=[1., 1., 1.],
            max_pixel_value=255.,
        ),
        ToTensorV2(),
    ]
)

In [ ]:
train_df, valid_df = train_test_split(df, random_state=2103, test_size=.2)

train_dataset = CarvanaDataset(train_df, train_transforms)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

valid_dataset = CarvanaDataset(valid_df, val_transforms)
valid_loader = DataLoader(valid_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

In [ ]:
# Check data
dataiter = iter(train_loader)
images, masks = dataiter.next()

print(images.shape)
print(masks.shape)

In [ ]:
# Visualize some groundtruth
rand_idx = np.random.randint(len(train_dataset), size=5)

fig, axes = plt.subplots(5, 2, figsize=(15, 20))

# Plot the images
for i, idx in enumerate(rand_idx):
    img, mask = train_dataset[idx]
    img = img.numpy()
    mask = mask.numpy()
    img = img.transpose(1, 2, 0)
    
    ax_img = axes[i][0]
    ax_mask = axes[i][1]
    
    ax_img.imshow(img)
    ax_img.get_xaxis().set_visible(False)
    ax_img.get_yaxis().set_visible(False)
    ax_img.set_title('original')
    
    ax_mask.imshow(mask)
    ax_mask.get_xaxis().set_visible(False)
    ax_mask.get_yaxis().set_visible(False)
    ax_mask.set_title('mask')

# Model

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
from torch.nn.init import kaiming_uniform_, xavier_uniform_

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True),
        )
        kaiming_uniform_(self.conv[0].weight, nonlinearity='relu')
        kaiming_uniform_(self.conv[3].weight, nonlinearity='relu')

    def forward(self, x):
        return self.conv(x)


class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of Unet
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of Unet
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2),
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            
            if x.shape != skip_connection.shape:
                x = TF.resize(x, skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx + 1](concat_skip)

        return self.final_conv(x)
    

model = UNET(3, 1)
model

# Train

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
def train_fn(loader, model, optimizer, scheduler, loss_fn, scaler):
    loop = tqdm(loader)

    for _, (data, targets) in enumerate(loop):
        data = data.float().to(DEVICE)
        targets = targets.float().unsqueeze(1).to(DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

In [ ]:
!mkdir 'predict_with_best_dice_score/'
!mkdir 'predict_with_best_loss/'
!mkdir 'runs/'

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=.75, patience=5, verbose=5, eps=1e-6)

scaler = torch.cuda.amp.GradScaler()

train_accuracy = []
train_dice_score = []
train_loss = []
valid_accuracy = []
valid_dice_score = []
valid_loss = []

best_dice_score = 0
best_loss = np.Inf

model.to(DEVICE)

for epoch in range(NUM_EPOCHS):
    print(f'epoch: {epoch}/{NUM_EPOCHS}')
    train_fn(train_loader, model, optimizer, scheduler, loss_fn, scaler)
    
    # Check accuracy and loss
    print('TRAIN:\t', end='')
    train_acc, train_d_score, train_l = check_accuracy(train_loader, model, loss_fn, DEVICE)
    train_accuracy.append(train_acc)
    train_dice_score.append(train_d_score)
    train_loss.append(train_l)
    
    print('VALID:\t', end='')
    valid_acc, valid_d_score, valid_l = check_accuracy(valid_loader, model, loss_fn, DEVICE)
    valid_accuracy.append(valid_acc)
    valid_dice_score.append(valid_d_score)
    valid_loss.append(valid_l)
    
    # Updates Learning Rate wrt the Average Validation Loss
    scheduler.step(valid_loss[-1])
    
    if best_dice_score < valid_d_score:
        print(f'\tSave best dice score: {best_dice_score} --> {valid_d_score}')
        best_dice_score = valid_d_score
        save_checkpoint(model, optimizer, file_name='best_dice_score.pt')
        save_predictions_as_imgs(valid_loader, model, folder='predict_with_best_dice_score/', device=DEVICE)
        
    if best_loss > valid_loss[-1]:
        print(f'\tSave best loss: {best_loss} --> {valid_loss}')
        best_loss = valid_loss[-1]
        save_checkpoint(model, optimizer, file_name='best_loss.pt')
        save_predictions_as_imgs(valid_loader, model, folder='predict_with_best_loss/', device=DEVICE)
    
    show_predictions(valid_dataset[0][0], valid_dataset[0][1], device=DEVICE)

In [ ]:
# test_img = np.array(Image.open('./train/e8f607100c1f_14.jpg').convert('RGB'))
# print(test_img.shape)

In [ ]:
# !unzip ../input/carvana-image-masking-challenge/sample_submission.csv.zip

In [ ]:
# sub = pd.read_csv('./sample_submission.csv')

# sub